In [1]:
from TranscriptionDataset.ArrangementDataset import ArrangementDataModule
from TranscriptionModel.Tuning import TuningModel
from pytorch_lightning import Trainer
import torch
from torch import nn
import os
import logging
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [12, 8]
print(f"Cuda : {torch.cuda.is_available()}")

Cuda : True


In [5]:
SAMPLE_RATE = 16000
dataset = "Trainsets/massive_test2.hdf5"

In [6]:
data_module = ArrangementDataModule(
    location=dataset,
    sample_rate=SAMPLE_RATE,
    test_size=0.1,
    num_workers=4,
    batch_size=4
)

Creating OneHotEncodings: 100%|██████████| 2036/2036 [00:01<00:00, 1634.71it/s]


In [7]:
model = TuningModel(
        tuning_size=data_module.transform.tuning_output_size,
        capo_size=data_module.transform.capo_output_size,
        arrangement_size=data_module.transform.arrangement_output_size
)

In [8]:
# train
trainer = Trainer(accelerator="gpu", devices=1)
trainer.fit(model, data_module)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\ritwi\anaconda3\envs\music\lib\site-packages\pytorch_lightning\loops\utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: C:\Users\ritwi\Github\MusicTranscription\lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name          | Type        | Params
-----------------------------------------------
0  | conv1         |

Sanity Checking: 0it [00:00, ?it/s]

RuntimeError: 0D or 1D target tensor expected, multi-target not supported